In [6]:
import numpy as np
import random
import scipy.optimize as opt
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

In [1]:
class Partition:
    _SAMPLING_NUM = 20
    
    PRINT_LOGIC = False
    
    def __init__(self, relation, x, y, sample_num):
        self.relation = relation
        self.x = x
        self.y = y
        self.sample_num = sample_num

        # initial param.
        self.m, self.b, self.k = self._init_param()
        self.param = 0
        self.func = getattr(self, '_'+relation)
        if self.PRINT_LOGIC:
            print("growth type : ", relation)
        
    def _init_param(self):
        m = random.uniform(1, 10)
        m = round(m, 4) * random.choice([1, -1])

        b = random.uniform(-10, 10)
        b = round(b, 4)

        k = random.uniform(1, 5)
        k = round(k, 4) * random.choice([1, -1])
        
        return m, b, k

    def params(self):
        if self.func == self._linear: # linear
            p0ini = [self.b, self.m]
            popt, pcov = curve_fit(lambda x, b, m: self.func(x, b, m), self.x, self.y, p0=p0ini, maxfev=30000)
        else:
            p0ini = [self.b, self.k]
            if self.func == self._exponential:
                a = random.uniform(2, 20)
                a = round(a, 4)
            else:
                a = random.uniform(0.85*self.x[0], 0.99*self.x[0])
                a = round(a, 4)
            
            popt, pcov = curve_fit(lambda x, b, k: self.func(x, a, b, k), self.x, self.y, p0=p0ini, maxfev=30000)            
            popt = np.insert(popt, 0, a)
        
        self.param = popt

        return popt, pcov
    
    # relation으로 param으로 초기점을 지나는 xfit, yfit을 구함
    def points(self, relation, params):
        xfit = np.linspace(self.x[0], self.x[1], self.sample_num)
        yfit = self.func( xfit, *params )

        points_ = self._delete_over_value(xfit, yfit)

        return list(points_)

    def _linear(self, x, b, m):
        return m*x + b

    def _log(self, x, a, b, k):
        return k*np.log(x-a) + b

    def _exponential(self, x, a, b, k):
        return k*np.exp(a*x) + b
        
    def sign(self):
        return np.sign(self.param[-1])
        
    
    #통과하는 두 점을 기준으로 양쪽을 자름
    def _delete_over_value(self, xfit, yfit):
        s = self.x[0]
        e = self.x[1]

        points = list(zip(xfit, yfit))

        points = [(x,y) for (x,y) in points if x>=s and x<=e]

        return points

    